In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import Bio
from Bio import SeqIO
from pandas import Series
import argparse
import re

In [2]:
chrlist = [''.join(['chr', str(i)]) for i in list(range(1, 23)) + ['X', 'Y']]

fa = dict()
for record in SeqIO.parse('/Users/yzli/Desktop/weilab/reference/hg38_genome.fa','fasta'):
    if record.id in chrlist:
        fa[record.id] = record

In [13]:
#forward strand sgRNA

In [4]:
nosp = pd.read_csv('../data/thr/nosp_t_genpos.txt',sep='\t')
sp1_23 = pd.read_csv('../data/thr/sp1_23_t_genpos.txt',sep='\t')
sp12_3 = pd.read_csv('../data/thr/sp12_3_t_genpos.txt',sep='\t')

In [5]:
sp1_23 = sp1_23[['chr','feature','strand','gene_name','transcript_id',
               'pro_id','base1','base2','base3','aa_genome_seq','aa_position']]
sp12_3 = sp12_3[['chr','feature','strand','gene_name','transcript_id',
               'pro_id','base1','base2','base3','aa_genome_seq','aa_position']]

In [6]:
nosp.loc[:,'type'] = 'nosp'
sp1_23.loc[:,'type'] = 'sp1_23'
sp12_3.loc[:,'type'] = 'sp12_3'

In [7]:
genpos_all = nosp.append(sp1_23).append(sp12_3)

In [8]:
genpos_all.shape

(809077, 12)

In [19]:
genpos_f = genpos_all[genpos_all['strand'] == '+']
genpos_r = genpos_all[genpos_all['strand'] == '-']

In [20]:
genpos_f.loc[:,'reg_from'] = genpos_f['base1'] + 14
genpos_f.loc[:,'reg_to'] = genpos_f['base1'] + 20

In [21]:
genpos_f.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to
0,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400070,18400071,18400072,ACA,23,nosp,18400084,18400090
1,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400076,18400077,18400078,ACT,25,nosp,18400090,18400096
2,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400241,18400242,18400243,ACC,80,nosp,18400255,18400261
3,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400244,18400245,18400246,ACA,81,nosp,18400258,18400264
4,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279


In [22]:
genpos_f.loc[:,'reg_seq'] = genpos_f.apply(
            lambda x: str(fa[x['chr']].seq[x['reg_from'] - 1: x['reg_to']]).upper(),
            axis=1)

In [23]:
genpos_f.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq
0,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400070,18400071,18400072,ACA,23,nosp,18400084,18400090,TCTTGAG
1,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400076,18400077,18400078,ACT,25,nosp,18400090,18400096,GCACCAG
2,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400241,18400242,18400243,ACC,80,nosp,18400255,18400261,TCAGACC
3,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400244,18400245,18400246,ACA,81,nosp,18400258,18400264,GACCACA
4,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT


In [24]:
genpos_f.shape

(414683, 15)

In [25]:
pam_f = []
for index, data in genpos_f.iterrows():
    for i in re.finditer('(?=GG)',data['reg_seq']):
        NGG_pos = i.span()[1]
        pam_f.append([data['chr'],data['strand'],data['gene_name'],
                      data['transcript_id'],data['pro_id'],data['base1'],
                      data['base2'],data['base3'],data['aa_genome_seq'],
                      data['aa_position'],data['type'],data['reg_from'],
                      data['reg_to'],data['reg_seq'],NGG_pos])

In [26]:
pam_f = DataFrame(pam_f,columns = ['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id',
       'base1', 'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type',
       'reg_from', 'reg_to', 'reg_seq', 'NGG_pos'])

In [27]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT,1
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT,2
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400460,18400461,18400462,ACA,153,nosp,18400474,18400480,AATCTGG,5
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400772,18400773,18400774,ACT,257,nosp,18400786,18400792,AGAGGTT,3
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400868,18400869,18400870,ACT,289,nosp,18400882,18400888,GGAACAA,0


In [28]:
pam_f.shape

(176249, 15)

In [29]:
pam_f.loc[:,'distance'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - pam_f['base1']

In [30]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT,1,14
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT,2,15
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400460,18400461,18400462,ACA,153,nosp,18400474,18400480,AATCTGG,5,18
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400772,18400773,18400774,ACT,257,nosp,18400786,18400792,AGAGGTT,3,16
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400868,18400869,18400870,ACT,289,nosp,18400882,18400888,GGAACAA,0,13


In [31]:
pam_f.loc[pam_f['distance'].isin([14,15,16]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 19
pam_f.loc[pam_f['distance'].isin([14,15,16]),'sg_to'] = pam_f['sg_from'] + 19 - 1

pam_f.loc[pam_f['distance'].isin([13,17]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 20
pam_f.loc[pam_f['distance'].isin([13,17]),'sg_to'] = pam_f['sg_from'] + 20 - 1

pam_f.loc[pam_f['distance'].isin([18]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 21
pam_f.loc[pam_f['distance'].isin([18]),'sg_to'] = pam_f['sg_from'] + 21 - 1

pam_f['sg_from'] = pam_f['sg_from'].map(lambda x:('%.0f')%x).astype(int)
pam_f['sg_to'] = pam_f['sg_to'].map(lambda x:('%.0f')%x).astype(int)

In [32]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT,1,14,18400254,18400272
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT,2,15,18400255,18400273
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400460,18400461,18400462,ACA,153,nosp,18400474,18400480,AATCTGG,5,18,18400457,18400477
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400772,18400773,18400774,ACT,257,nosp,18400786,18400792,AGAGGTT,3,16,18400769,18400787
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400868,18400869,18400870,ACT,289,nosp,18400882,18400888,GGAACAA,0,13,18400861,18400880


In [33]:
pam_f.loc[:,'sg_seq'] = pam_f.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_from'] - 1: x['sg_to']]).upper(),
            axis=1)

In [34]:
pam_f.loc[:,'pam'] = pam_f.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] : x['sg_to'] + 3]).upper(),
            axis=1)

In [35]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT,1,14,18400254,18400272,TTCAGACCACAATGTTAGG,AGG
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,18400273,18400279,AGGGTAT,2,15,18400255,18400273,TCAGACCACAATGTTAGGA,GGG
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400460,18400461,18400462,ACA,153,nosp,18400474,18400480,AATCTGG,5,18,18400457,18400477,TTGACAGAAGAGAGAGGAATC,TGG
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400772,18400773,18400774,ACT,257,nosp,18400786,18400792,AGAGGTT,3,16,18400769,18400787,AAAACTCTCACTGAGGAAG,AGG
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400868,18400869,18400870,ACT,289,nosp,18400882,18400888,GGAACAA,0,13,18400861,18400880,ATCCCTTACTATTTAGAATA,AGG


In [36]:
pam_f.shape

(176249, 20)

In [37]:
pam_f.loc[:,'aa_id'] = pam_f['chr'].str.cat(pam_f['base1'].astype(str),sep='-')
pam_f.loc[:,'sg_id'] = pam_f['aa_id'].str.cat(pam_f['sg_seq'].astype(str),sep='-')

In [38]:
pam_f['aa_id'].unique().shape

(87473,)

In [39]:
pam_f['sg_id'].unique().shape

(117057,)

In [40]:
pam_f['sg_seq'].unique().shape

(114484,)

In [41]:
#reverse strand sgRNA

In [42]:
genpos_r.loc[:,'reg_from'] = genpos_r['base3'] - 20 
genpos_r.loc[:,'reg_to'] = genpos_r['base3'] - 14

In [43]:
genpos_r.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to
413643,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9115787,9115788,9115789,ACA,21,nosp,9115769,9115775
413644,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495
413645,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113501,9113502,9113503,ACC,43,nosp,9113483,9113489
413646,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113498,9113499,9113500,ACT,44,nosp,9113480,9113486
413647,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113459,9113460,9113461,ACA,57,nosp,9113441,9113447


In [44]:
genpos_r.loc[:,'reg_seq'] = genpos_r.apply(
            lambda x: str(fa[x['chr']].seq[x['reg_from'] - 1: x['reg_to']].reverse_complement()).upper(),
            axis=1)

In [3]:
genpos_r.head()

NameError: name 'genpos_r' is not defined

In [46]:
pam_r = []
for index, data in genpos_r.iterrows():
    for i in re.finditer('(?=GG)',data['reg_seq']):
        NGG_pos = i.span()[1]
        pam_r.append([data['chr'],data['strand'],data['gene_name'],
                      data['transcript_id'],data['pro_id'],data['base1'],
                      data['base2'],data['base3'],data['aa_genome_seq'],
                      data['aa_position'],data['type'],data['reg_from'],
                      data['reg_to'],data['reg_seq'],NGG_pos])

In [47]:
pam_r = DataFrame(pam_r,columns = ['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id',
       'base1', 'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type',
       'reg_from', 'reg_to', 'reg_seq', 'NGG_pos'])

In [48]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9115787,9115788,9115789,ACA,21,nosp,9115769,9115775,CTCTGGA,4
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495,GAAGGGC,3
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495,GAAGGGC,4
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113453,9113454,9113455,ACT,59,nosp,9113435,9113441,CTTGGAG,3
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113405,9113406,9113407,ACT,75,nosp,9113387,9113393,GGAGAAT,0


In [49]:
pam_r.shape

(167029, 15)

In [50]:
pam_r.loc[:,'distance'] = pam_r['base3'] - (pam_r['reg_to'] - pam_r['NGG_pos'] + 1)

In [51]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9115787,9115788,9115789,ACA,21,nosp,9115769,9115775,CTCTGGA,4,17
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495,GAAGGGC,3,16
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495,GAAGGGC,4,17
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113453,9113454,9113455,ACT,59,nosp,9113435,9113441,CTTGGAG,3,16
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113405,9113406,9113407,ACT,75,nosp,9113387,9113393,GGAGAAT,0,13


In [52]:
pam_r.loc[pam_r['distance'].isin([14,15,16]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 19
pam_r.loc[pam_r['distance'].isin([14,15,16]),'sg_to'] = pam_r['sg_from'] - 19 + 1

pam_r.loc[pam_r['distance'].isin([13,17]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 20
pam_r.loc[pam_r['distance'].isin([13,17]),'sg_to'] = pam_r['sg_from'] - 20 + 1

pam_r.loc[pam_r['distance'].isin([18]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 21
pam_r.loc[pam_r['distance'].isin([18]),'sg_to'] = pam_r['sg_from'] - 21 + 1

pam_r['sg_from'] = pam_r['sg_from'].map(lambda x:('%.0f')%x).astype(int)
pam_r['sg_to'] = pam_r['sg_to'].map(lambda x:('%.0f')%x).astype(int)

In [53]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9115787,9115788,9115789,ACA,21,nosp,9115769,9115775,CTCTGGA,4,17,9115792,9115773
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495,GAAGGGC,3,16,9113512,9113494
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495,GAAGGGC,4,17,9113512,9113493
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113453,9113454,9113455,ACT,59,nosp,9113435,9113441,CTTGGAG,3,16,9113458,9113440
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113405,9113406,9113407,ACT,75,nosp,9113387,9113393,GGAGAAT,0,13,9113414,9113395


In [54]:
pam_r.loc[:,'sg_seq'] = pam_r.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] - 1: x['sg_from']].reverse_complement()).upper(),
            axis=1)

In [55]:
pam_r.loc[:,'pam'] = pam_r.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] - 4 : x['sg_to'] - 1].reverse_complement()).upper(),
            axis=1)

In [56]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9115787,9115788,9115789,ACA,21,nosp,9115769,9115775,CTCTGGA,4,17,9115792,9115773,CCCACAGACGCCTCAGTCTC,TGG
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495,GAAGGGC,3,16,9113512,9113494,CACACTGAGACCACTGAGA,AGG
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113507,9113508,9113509,ACT,41,nosp,9113489,9113495,GAAGGGC,4,17,9113512,9113493,CACACTGAGACCACTGAGAA,GGG
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113453,9113454,9113455,ACT,59,nosp,9113435,9113441,CTTGGAG,3,16,9113458,9113440,GTGACTGTAAGTGCTTCCT,TGG
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113405,9113406,9113407,ACT,75,nosp,9113387,9113393,GGAGAAT,0,13,9113414,9113395,CCTCTTCACTGACCTGGAGG,CGG


In [57]:
pam_r.shape

(167029, 20)

In [58]:
pam_r.loc[:,'aa_id'] = pam_r['chr'].str.cat(pam_r['base3'].astype(str),sep='-')
pam_r.loc[:,'sg_id'] = pam_r['aa_id'].str.cat(pam_r['sg_seq'].astype(str),sep='-')

In [59]:
pam_r['aa_id'].unique().shape

(86477,)

In [60]:
pam_r['sg_id'].unique().shape

(115492,)

In [61]:
pam_r['sg_seq'].unique().shape

(113551,)

In [62]:
#sgRNA filter with 0.2 < GC content < 0.8 & remove TTTT

In [63]:
sg = pam_f.append(pam_r)

In [64]:
sg.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,...,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam,aa_id,sg_id
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,...,18400279,AGGGTAT,1,14,18400254,18400272,TTCAGACCACAATGTTAGG,AGG,chr8-18400259,chr8-18400259-TTCAGACCACAATGTTAGG
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,...,18400279,AGGGTAT,2,15,18400255,18400273,TCAGACCACAATGTTAGGA,GGG,chr8-18400259,chr8-18400259-TCAGACCACAATGTTAGGA
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400460,18400461,18400462,ACA,153,...,18400480,AATCTGG,5,18,18400457,18400477,TTGACAGAAGAGAGAGGAATC,TGG,chr8-18400460,chr8-18400460-TTGACAGAAGAGAGAGGAATC
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400772,18400773,18400774,ACT,257,...,18400792,AGAGGTT,3,16,18400769,18400787,AAAACTCTCACTGAGGAAG,AGG,chr8-18400772,chr8-18400772-AAAACTCTCACTGAGGAAG
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400868,18400869,18400870,ACT,289,...,18400888,GGAACAA,0,13,18400861,18400880,ATCCCTTACTATTTAGAATA,AGG,chr8-18400868,chr8-18400868-ATCCCTTACTATTTAGAATA


In [65]:
sg.loc[:,'GC_content'] = (sg['sg_seq'].str.count('G') + sg['sg_seq'].str.count('C')) / sg['sg_seq'].str.len()

In [66]:
sg_gc = sg[(sg['GC_content'] >= 0.2) & (sg['GC_content'] <= 0.8)]

In [67]:
sg_gc_4t = sg_gc.append(
    sg_gc[sg_gc['sg_seq'].str.contains('TTTT')]).drop_duplicates(keep=False)

In [68]:
sg_gc_4t.columns

Index(['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id', 'base1',
       'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type', 'reg_from',
       'reg_to', 'reg_seq', 'NGG_pos', 'distance', 'sg_from', 'sg_to',
       'sg_seq', 'pam', 'aa_id', 'sg_id', 'GC_content'],
      dtype='object')

In [69]:
sg_gc_4t = sg_gc_4t[['sg_id','aa_id','chr','strand','gene_name','transcript_id',
                    'pro_id','base1','base2','base3','aa_genome_seq','aa_position',
                    'type','sg_seq','sg_from','sg_to','pam','GC_content']]

In [70]:
sg_gc_4t.head()

,sg_id,aa_id,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,sg_seq,sg_from,sg_to,pam,GC_content
0,chr8-18400259-TTCAGACCACAATGTTAGG,chr8-18400259,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,TTCAGACCACAATGTTAGG,18400254,18400272,AGG,0.421053
1,chr8-18400259-TCAGACCACAATGTTAGGA,chr8-18400259,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400259,18400260,18400261,ACC,86,nosp,TCAGACCACAATGTTAGGA,18400255,18400273,GGG,0.421053
2,chr8-18400460-TTGACAGAAGAGAGAGGAATC,chr8-18400460,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400460,18400461,18400462,ACA,153,nosp,TTGACAGAAGAGAGAGGAATC,18400457,18400477,TGG,0.428571
3,chr8-18400772-AAAACTCTCACTGAGGAAG,chr8-18400772,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400772,18400773,18400774,ACT,257,nosp,AAAACTCTCACTGAGGAAG,18400769,18400787,AGG,0.421053
4,chr8-18400868-ATCCCTTACTATTTAGAATA,chr8-18400868,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400868,18400869,18400870,ACT,289,nosp,ATCCCTTACTATTTAGAATA,18400861,18400880,AGG,0.250000


In [71]:
sg_gc_4t['sg_id'].unique().shape

(223303,)

In [72]:
sg_gc_4t['aa_id'].unique().shape

(167414,)

In [73]:
sg_gc_4t['sg_seq'].unique().shape

(217211,)

In [74]:
sg_gc_4t.to_csv('../data/thr/sg_thr.txt',sep='\t',index=False)